# Statistical Machine Learning: Project Simulations

Author: Pascal Dreieck

## Set Up

In [ ]:
# Import

import os
import sys

import pickle as pk

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification, make_blobs

from optimisation_algorithms import logistic_f, logistic_df, GDClassifer, SGDClassifer, SVRGClassifer, SVRGClassifer_Alt, SAGAClassifer

RANDOM_STATE = 42

In [ ]:
# Datasets
N_SAMPLES = 50
DIMENSIONS = [10,10,10]

DATASETS = []
for i, dim in enumerate(DIMENSIONS):
    cluster_1 = np.random.multivariate_normal(
        mean=[1] * dim,
        cov=np.diag([2] * dim),
        size=N_SAMPLES * dim
        )
    cluster_2 = np.random.multivariate_normal(
        mean=[-1] * dim,
        cov=np.diag([2 * np.sqrt(10)] * dim),
        size=N_SAMPLES * dim
        )
    Xn = np.concatenate((cluster_1, cluster_2), axis=0)

    Yn = np.ones((N_SAMPLES * dim * 2,))
    Yn[:N_SAMPLES * dim] = -1

    DATASETS.append((Xn, Yn))

# Redo X2 to vary covariance
cluster_1 = np.random.multivariate_normal(
    mean=[1] * DIMENSIONS[-1],
    cov=np.diag(([4] * (DIMENSIONS[-1] // 2)) + ([1] * (DIMENSIONS[-1] - (DIMENSIONS[-1] // 2 )))),
    size=N_SAMPLES * DIMENSIONS[-1]
    )
cluster_2 = np.random.multivariate_normal(
    mean=[-1] * DIMENSIONS[-1],
    cov=np.diag(([1] * (DIMENSIONS[-1] // 2)) + ([4] * (DIMENSIONS[-1] - (DIMENSIONS[-1] // 2 )))),
    size=N_SAMPLES * DIMENSIONS[-1]
    )
new_X2 = np.concatenate((cluster_1, cluster_2), axis=0)

(old_X2, Y2) = DATASETS[-1]
DATASETS[-1] = (new_X2, Y2)

# Center at (0,0)
for Xn, Yn in DATASETS:
    Xn[:, 0] = Xn[:, 0] - Xn[:, 0].mean()
    Xn[:, 1] = Xn[:, 1] - Xn[:, 1].mean()

## Tests

In [ ]:
# TEST VALUES
ETAS = (5e-1, 1e-1, 5e-2, 1e-2)
MAX_ITER  = 200 # 1000
MAX_GRADS = 10000
w_0s = []
for dim in DIMENSIONS:
    temp = np.array([0] * dim)
    w_0s.append(temp)

#### Step Size Comparison

In [ ]:
# Rerun tests or read from file
RERUN_TESTS = True

In [ ]:
if RERUN_TESTS:
    # Setup dictionaries
    step_size_complexity_dict = {}

    # Run over all datasets
    for j, tup in enumerate(DATASETS):
        print(f"Dataset {j+1} / {len(DATASETS)}")
        Xn, Yn = tup
        M = len(Xn) // 2

        for i, eta in enumerate(ETAS):
            print(f"Eta {i+1} / {len(ETAS)}")
            
            print("gd...", end="")
            step_size_complexity_dict[(j, i, 'gd')] = GDClassifer(logistic_f, logistic_df, Xn, Yn, w_0s[j],
                eta=eta, stopping_times=(None,MAX_GRADS), random_state=RANDOM_STATE)
            print("sgd...", end="")
            step_size_complexity_dict[(j, i, 'sgd')] = SGDClassifer(logistic_f, logistic_df, Xn, Yn, w_0s[j],
                eta=eta, stopping_times=(None,MAX_GRADS), random_state=RANDOM_STATE)
            print("saga...", end="")
            step_size_complexity_dict[(j, i, 'saga')] = SAGAClassifer(logistic_f, logistic_df, Xn, Yn, w_0s[j],
                eta=eta, stopping_times=(None,MAX_GRADS), random_state=RANDOM_STATE)
            print("svrg...", end="")
            step_size_complexity_dict[(j, i, 'svrg')] = SVRGClassifer(logistic_f, logistic_df, Xn, Yn, w_0s[j],
                eta=eta, m=M, stopping_times=(None,MAX_GRADS), random_state=RANDOM_STATE)
            # print("svrg_alt...", end="")
            # step_size_complexity_dict[(j, i, 'svrg_alt')] = SVRGClassifer_Alt(logistic_f, logistic_df, Xn, Yn, w_0s[j],
            #     eta=eta, m=M, stopping_times=(None,MAX_GRADS), random_state=RANDOM_STATE)
            print("")

In [ ]:
if RERUN_TESTS:
    # Save data
    with open('data/step_size_complexity_dict.pk','wb') as f:
        pk.dump(step_size_complexity_dict, f)

In [ ]:
if not RERUN_TESTS:
    # Read values
    with open('data/step_size_complexity_dict.pk','rb') as f:
        data = f.read()
        step_size_complexity_dict = pk.loads(data)

### Plot

In [ ]:
# Plotting
colours = ["black", "red", "violet", "darkblue", 'green']
linestyles = ['solid', 'dashdot', 'dashed', 'dotted']

#### Step-Size

In [ ]:
# Setup plot
fig = plt.figure(figsize=(10, 8), constrained_layout=True)
fig.suptitle(f"Convergence of Optimisation Algorithms", fontsize=20, x=0.5, y=1.05)

subfigs = fig.subfigures(nrows=len(ETAS[:-1]), ncols=1)

for i, eta in enumerate(ETAS[:-1]):
    subfigs[i].suptitle(f"$\eta$ = {eta}", fontsize=15, x=-0.015, y=0.65, rotation=90)

    # Run over all datasets
    for j, _ in enumerate(DATASETS):

        # Complexity Subplot
        ax = subfigs[i].add_subplot(1, len(DATASETS), j + 1)

        _, gd_loss, _, gd_grad_count = step_size_complexity_dict[(j, i, 'gd')]
        _, sgd_loss, _, sgd_grad_count = step_size_complexity_dict[(j, i, 'sgd')]
        _, svrg_loss, _, svrg_grad_count = step_size_complexity_dict[(j, i, 'svrg')]
        # _, svrg_alt_loss, _, svrg_alt_grad_count = step_size_complexity_dict[(j, i, 'svrg_alt')]
        _, saga_loss, _, saga_grad_count = step_size_complexity_dict[(j, i, 'saga')]

        def helper(grad_count, loss, max_grads):
            grad_output = []
            for grad in grad_count:
                if grad > max_grads:
                    break
                else:
                    grad_output.append(grad)
            loss_output = loss[:len(grad_output)]

            return grad_output, loss_output
        
        DIV = 1
        ax.plot(*helper(gd_grad_count, gd_loss, MAX_GRADS//DIV),
            label=f"gd" if i == 0 and j == 0 else None,
            c=colours[0], linestyle=linestyles[0], linewidth=2.0)
        ax.plot(*helper(sgd_grad_count, sgd_loss, MAX_GRADS//DIV),
            label=f"sgd" if i == 0 and j == 0 else None,
            c=colours[1], linestyle=linestyles[0], linewidth=2.0)
        ax.plot(*helper(svrg_grad_count, svrg_loss, MAX_GRADS//DIV),
            label=f"svrg" if i == 0 and j == 0 else None,
            c=colours[3], linestyle=linestyles[0], linewidth=2.0)
        # ax.plot(svrg_alt_grad_count, svrg_alt_loss,
        #     label=f"svrg_alt: eta = {eta}" if j == 0 else None,
        #     c=colours[4], linestyle=linestyles[0], linewidth=2.5)
        ax.plot(*helper(saga_grad_count, saga_loss, MAX_GRADS//DIV),
            label=f"saga" if i == 0 and j == 0 else None,
            c=colours[4], linestyle=linestyles[0], linewidth=2.0)

        if i == 0:
            ax.set_title(f"Dataset {j}", fontsize=15)
        ax.set_xlabel("Gradients Computed", weight='bold', fontsize=10)
        ax.set_ylabel("Training Loss", weight='bold', fontsize=10)
        ax.set_yscale("log")
        ax.tick_params(which='major', size=10)
        ax.tick_params(which='minor', size=7)
        ax.tick_params(which='minor', )

# Format and Save Plot
fig.legend(loc="lower center", fontsize=15, bbox_to_anchor=(0.5, -0.075), labelspacing=1.0, ncols = 4)
fig.savefig(f"plots/total_plot.png", bbox_inches='tight')
plt.figure()